In [1]:
#!pip install pandas
#!pip install seaborn
#!pip install openpyxl

In [2]:
import pandas as pd
import numpy as np
import time
import csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import model_from_json
from tensorflow import data
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf

In [3]:
path_name_results='../results/'
file_result = 'Result_LSTM_TSLA_stock_prices.csv'

In [4]:
# Define the stock symbol (Tesla)
stock_symbol = "TSLA"

# Define the start and end dates for the historical data
start_date = "2017-01-01"
end_date = "2023-09-21"

# Download historical data
dataset = yf.download(stock_symbol, start=start_date, end=end_date)

#dataset.columns = ['year','month', 'date','total_sunspot_number','std_derivation','num_observations','def_prov_indicator']
#dataset.values

ds=pd.DataFrame()
#ds['date']=dataset['Open'].index
ds['num_observations']=dataset['Open'].values
dataset=ds

[*********************100%%**********************]  1 of 1 completed


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1690 entries, 0 to 1689
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_observations  1690 non-null   float64
dtypes: float64(1)
memory usage: 13.3 KB


In [6]:
#checks if there are null variables
dataset.isna().sum()

num_observations    0
dtype: int64

In [7]:
def salvar_resultado(nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration):
  #Script to write training cycle results
  data = [nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration]
  fields = ['Dataset','Best Params','n_time_steps','MSE', 'RMSE', 'MAE', 'MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['Dataset','Best Params','n_time_steps','MSE', 'RMSE', 'MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)    

In [8]:
# convert an array of values into a dataset matrix
def create_matrix_dataset(dataset, look_back=1):
 dX, dY = [], []
 for i in range(len(dataset)-look_back-1):
  a = dataset[i:(i+look_back), 0]
  dX.append(a)
  dY.append(dataset[i + look_back, 0])
 return np.array(dX), np.array(dY)

In [9]:
  
def save_model(model,n_time_steps):
  # serialize model to JSON
  model_json = model.to_json()
  with open(f'{path_name_results}model_{n_time_steps}.json', "w") as json_file:
    json_file.write(model_json)

  # serialize weights to HDF5
  model.save_weights(f'{path_name_results}model_{n_time_steps}.h5')
  print("Saved model to disk")


In [10]:
def gera_resultado(y_test, predict,nm_dataset, resultado, n_time_steps, Duracao):
 #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
 MSE = mean_squared_error(y_test, predict)    
 #Square Root of Mean Error - RMSE
 RMSE = np.sqrt(mean_squared_error(y_test, predict))    
 #Mean Absolute Distance or Mean Absolute Error - MAE
 MAE= median_absolute_error(y_pred=predict, y_true = y_test) 
  
 #Calculate the MAPE (Mean Absolute Percentage Error)
 MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100   
  
 sMAPE = round(
 	np.mean(
 		np.abs(predict - y_test) /
 		((np.abs(predict) + np.abs(y_test)))
 	)*100, 2
 ) 
 salvar_resultado(nm_dataset, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duracao)

In [11]:
def previsao_LSTM(nm_dataset, dataset, n_time_steps, l1, l2, l3,num_epochs, batch_size ): 
 #num_epochs = 100 # number of epochs for train
 #n_time_steps = 6
 #l1, l2, l3 = 32 , 64, 32  # 4.61 smape   
 #l1, l2, l3 = 64 , 64, 64  # 4.61 smape
 #nm_dataset ='sunspot'
 #Split dataset in treinam /  80% treinam  20% test
 dataset=np.array(dataset) 
 nlinhas = int(len(dataset) * 0.80)
 test = dataset[nlinhas:len(dataset),:]  
 train = dataset[0:nlinhas,:] 
 #  reshape into X=t and Y=t+1 ot n_time_steps by steps
 #n_time_steps = 5
 X_train, Y_train = create_matrix_dataset(train, n_time_steps)
 X_test, Y_test = create_matrix_dataset(test, n_time_steps) 
 #X_train.shape , Y_train.shape , X_test.shape , Y_test.shape  
 #reshape input to be [samples, time steps, features]
 X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
 X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1)) 
 # Stores the training execution start time
 Hora_Inicio = time.time()
   
 resultado = "LSTM (" + str(l1) + "," + str(l2) + "," + str(l3) + ") n_time_steps=" + str(n_time_steps) + str(" epochs=") + str(num_epochs)
 print(resultado)
 # create and fit the LSTM network

 steps_per_epoch = len(X_train) 
 model = Sequential()
 model.add(LSTM(l1, batch_input_shape=(batch_size, n_time_steps, 1 ), stateful=True, return_sequences=True))
 model.add(LSTM(l2, batch_input_shape=(batch_size, n_time_steps, 1 ), stateful=True, return_sequences=True))
 model.add(LSTM(l3, batch_input_shape=(batch_size, n_time_steps, 1 ), stateful=True))
 model.add(Dense(1))
 model.compile(loss='mean_squared_error', optimizer='adam',run_eagerly=True)
 
 #equalize train data to be multiple of batch_size
 train_size = int(np.trunc(len(X_train) / batch_size))
 X_train = X_train[0:(train_size * batch_size),:]
 Y_train = Y_train[0:(train_size * batch_size)]
 
 #equalize test data to be multiple of batch_size
 test_size = int(np.trunc(len(X_test) / batch_size))
 X_test = X_test[0:(test_size * batch_size),:]
 Y_test = Y_test[0:(test_size * batch_size)]
 
 for i in range(num_epochs):	
   model.fit(X_train, Y_train, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
   model.reset_states()
 
 #predict values to X_test with size multiple of batch_size
 predict = model.predict(X_test, batch_size=batch_size)   
 
 Y_test = [Y_test]
 predict = predict.reshape(-1,1)[0:batch_size]   
 Y_test=np.array(Y_test[0][0:batch_size]).reshape(batch_size,1) 
 
 Hora_Fim = time.time()   
 #Calculate the duration of the training execution
 Duracao = Hora_Fim - Hora_Inicio   
 
 #calc metrics of error and save in file
 gera_resultado(Y_test, predict,nm_dataset, resultado, n_time_steps, Duracao)



In [12]:
        
#create file to results
criar_arquivo_resultado()

print('forecast for TSLA Stock prices')
num_epochs = 100 # number of epochs for train
batch_size = 24

l1, l2, l3 = 8, 18, 8    # 91.28382852134614, 83.97, 1334.49072265625
l1, l2, l3 = 90, 90, 90    # 93.12439093404133, 87.14, 1097.8796606063843
l1, l2, l3 = 88, 88, 98   # 
for n_time_steps in range(1,13): #predict with 1 to 12 past values of medition 
    previsao_LSTM('TSLA', dataset, n_time_steps, l1, l2, l3,num_epochs, batch_size)

#
def random_model():
  for n_time_steps in range(1,2): #predict with 1 to 12 past values of medition    
    for l1 in range(8,101,40): # chose layer 1 nodes - min 2 and max 4
        for l2 in range(8,101,20): # chose layer 2 nodes - min 4 and max 12
            for l3 in range(8,101,10): # chose layer 3 nodes - min 6 and max 8
                previsao_LSTM('TSLA', dataset, n_time_steps, l1, l2, l3,num_epochs, batch_size)    
                
#random_model()

forecast for TSLA Stock prices
LSTM (88,88,98) n_time_steps=1 epochs=100
56/56 [==============================] - 9s 160ms/step - loss: 2286.1497
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['TSLA', 'LSTM (88,88,98) n_time_steps=1 epochs=100', 1, 45964.805937724836, 214.39404361531325, 216.09769439697266, 91.92419790957501, 85.06, 701.099692106247]
LSTM (88,88,98) n_time_steps=2 epochs=100
56/56 [==============================] - 5s 82ms/step - loss: 2137.0330
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['TSLA', 'LSTM (88,88,98) n_time_steps=2 epochs=100', 2, 15155.642824847586, 123.10825652590319, 124.9817886352539, 52.55823928168426, 35.69, 731.8308744430542]
LSTM (88,88,98) n_time_steps=3 epochs=100
56/56 [==============================] - 9s 158ms/step - loss: 1948.1517
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['TSLA', 'LSTM (88,88